<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div> 

<br><br><br><br>

# Simulation of dose-response curves of agonists using affinity values

In this tutorial we will simulate a mathematical model of a signaling pathway to obtain dose-response curves, from wich  *potency (EC<sub>50</sub>)* values of agonists can be infered. 


This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial1.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>

</div>  


## Installing the SSB toolkit library

In [ ]:
!pip install --quiet ssbtoolkit

Run the following if you are working on Google colab

In [ ]:
import os
os.kill(os.getpid(), 9)

### Importing dependencies

In [ ]:
import ssbtoolkit as ssb 
import numpy as np
import pandas as pd
import os

Run the following command if you are working on the Ebrains platform

In [ ]:
os.environ['BNGPATH'] = '/opt/app-root/src/.local/lib/python3.8/site-packages/bionetgen/bng-linux/'

## Loading experimental data

Once the SSBtoolkit environment is set up we are ready to start to simulate.

We will begin by loading the affinity data of some selective agonists of the Adenosine 2A receptor. The experimental affinity data was taken from *([Varani, K. et al., 2015](https://doi.org/10.1021/acs.jmedchem.5b00215))*. This data can be directly downloaded from the repository.

In [ ]:
!curl -s -O https://raw.githubusercontent.com/rribeiro-sci/SSBtoolkit/main/example/A2AR_binding_data.csv

In [ ]:
data = pd.read_csv('A2AR_binding_data.csv')
data

Commonly, the experimental affinity values come in different "flavors". 

The SSB framework just accepts affinity values as pK<sub>d</sub>. Since the affinity values in our data set is repported as K<sub>i</sub> and EC<sub>50</sub> and we need to convert them to pK<sub>i</sub> and pEC<sub>50</sub> repectively.


In [ ]:
data['pKi'] = data.Ki.apply(lambda x: -np.log10(x*1E-6))
data['pEC50']=data.EC50.apply(lambda x: -np.log10(x*1E-6))
data

## Selection of the signaling pathway 

The core of the SSB framework is, naturally, the mathematical models of the GPCRs' signaling pathways. 

Since G-protein sub-families are classified by their $\alpha$ subunits, this classfication as been served to identify their signaling pathways:
* G<sub>s</sub>
* G<sub>i/o</sub>
* G<sub>q/11</sub>
* G<sub>12/13</sub>

📖 See [The SSB toolkit](https://ssbtoolkit.readthedocs.io/) documentation for more details.

We can define manualy the G$\alpha$ pathway we want to work with, or simply query our internal database of human GPCR pathways using the UNIPROT id of our receptor using the SSBtoolkit built-in function `Utils.GetGProtein()`. The UNIPROT id for the human Adenosine A2 receptot is [P29274](https://www.uniprot.org/uniprot/P29274).

In [ ]:
uniprotID = 'P29274'
gprotein = ssb.Utils.GetGProtein(uniprotID)
gprotein

<span style="color:black">⚠️ WARNING: our toolkit was specifically design for human GPCRs. The quering for pathways for GPCRs other than Human may be included in the future. However, if you want to study a non-human GPCR you can still use the SSB toolkit by setting manually the G$\alpha$ pathway.</span>


##  Preparation, Simulation and Analysis

To obtain a dose-response curve from the simulation of signaling pathways, individual simulations of the pathway according to a series of ligand concentrations must be performed (as it would be done in the wet-lab).  

To define an array of ligand concentrations we will use a geometric progression. The reason why we use a geometric progression is due to the impact of the dilution fraction on the accuracy of K<sub>d</sub> and EC<sub>50</sub>/IC<sub>50</sub> values experimentally estimated. This factor, that defines the spacing between the adjacent concentration values, has an impact on the concentration values that are on the linear portion of the curve. Therefore, using a geometric progression we can mimic the experimental conditions where each concentration equals to the power of 2 of the previous lowest concentration *([Sebaugh, J.L., 2011](https://doi.org/10.1002/pst.426))*

<span style="color:black"> ⚠️ WARNING: the SSB toolkit uses μM as default concentration units.</span>


In [ ]:
# setting the range of ligand concentrations
lig_conc_min = 1E-4 # μM
lig_conc_max = 1E4  # μM
lig_conc_range = np.geomspace(lig_conc_min, lig_conc_max, 20) # 20 concentration values

SSB simulations are also sensible to the concentration of the protein, wich must be, again, in <b><i>μM</i></b>. However, the  concentration values reported in functional assays comes, commonly, in <b><i>μg</i></b> of protein.

The experimental data we are using was obtained from a functional assay using 50 <b><i>μg</i></b> of protein. We can use the SSBtoolkit built-in function `Utils.MicrogramsToNanomolar()` to convert  <b><i>μg</i></b> of protein in <b><i>μM</i></b>.

In [ ]:
# the following function takes as input the UniProtID and the concentration in μg.
prot_conc = ssb.Utils.MicrogramsToNanomolar(uniprotID, 50) 
print(prot_conc, 'μM')

Finnaly, we need to define the length and number of time steps (resolution) of the simulation. 

In [ ]:
time = 10000  # time of simulation in seconds
nsteps = 1000   # number of time steps

## Integration of ODEs 

After having defined all the simulation parameters we are ready to proceed with the simulations. A simulation of a methamatical model of a signaling pathway consists of the integration of a set of ordinary differential equations (ODEs) as function of time. Since each ODE represents a molecular event of the signaling pathway, when integrated over time, such equations can describe how the concentration of species inside the model changes over time. The key point of this tutorial is the use of the drug-receptor affinity value (K<sub>d</sub>) to fire up the model. With the K<sub>d</sub> values one can calculate the fraction of receptors that are occupied by the ligand in the equilibrium and, according to the *occupancy theory*, the fraction of occupied receptors represents the concentration of activated receptors in the equilibrium *([Kenakin T., 2004 ](https://doi.org/10.1016/j.tips.2004.02.012))*. 📖 Read the [Docs](https://ssbtoolkit.readthedocs.io/) for more details.


In this tutorial we want to simulate dose-response curves of agonists towards Adenosine A2 receptor, so, we will use the SSBtoolkit built-in class `Simulation.Activation()`. 


<span style='color:black'>ℹ️ If you want study antagonists follow the tutorial [Simulation of dose-responses curves of antagonists](https://github.com/rribeiro-sci/SSBtoolkit/blob/main/SSBtoolkit_Tutorial2.ipynb).</span>

In [ ]:
# we will start by creating a simulation instance.
sim = ssb.Simulation.Activation()

In [ ]:
# configuration of simulation parameters
sim.SetSimulationParameters(ligands=data.id.to_list()[:2], 
                            affinities=data.pKi.to_list()[:2], 
                            pathway=gprotein, 
                            receptor_conc=prot_conc,
                            lig_conc_range=lig_conc_range, 
                            ttotal=time, 
                            nsteps=nsteps,  
                            binding_kinetics=False)

In [ ]:
#Start the simulation
sim.Run()

In the end, the concentration values of the species of the signaling pathway over the simulation time will be saved inside the instance.

The response of a signaling pathway is, naturally, represented by the increase or decrease of one of the species described by the model. So, to predict the dose-response curve we need, firstly, to extract the maximum concentration value orbserved for one specie from each individual simulation (from the series of simulations for each ligand concentration). Then, such values will be fitted to a logistic regression. 

To achieve this, we will use the analysis function:

In [ ]:
sim.Analysis()

We can now to plot the dose-response curves:

In [ ]:
sim.ShowCurve()

Finnaly, from the dose-response curves we can interpolate the EC<sub>50</sub> values.

In [ ]:
sim.ShowPotency()

💡 The potency predicted values can be exported as a python dictionary using the function `sim.PotencyToDict()` or saved in a csv file: `sim.PotencyToCSV()`. 


## Congratulations! 

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with SSBtoolkit, we encourage you to finish the rest of the tutorials in this series. 

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={{SSB} toolkit: from molecular structure to subcellular signaling pathways.},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={bioRxiv},
    url={https://www.biorxiv.org/content/10.1101/2022.11.08.515595v1},
    doi={10.1101/2022.11.08.515595},
    year={2022}
}
```



## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  